In [ ]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
home = '/Users/mahtin'

somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "testpon")

static = os.path.join(home, "Dropbox/Icke/Work/static")

## create EBmatrix from different input
+ filterVar.mawk is the input fork

In [ ]:
from ebraw import tumor2matrix

EBconfig = {
    "pon_path": pon_path,
    "mawk_path": "../shell",
    "genome_split": f"{static}/genome/gatk/hg38/split",
    "pon_path": os.path.join(testdata, "testpon"),
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"},
    "use_cache": True
}


bam_file = f"{testdata}/bam/002_A.bam"
pileup_file = os.path.join(ebdata, "002_A-B.pileup.gz")
mut_file = os.path.join(ebdata, "002.csv")
pon_list = "Pon_chr7.txt"
chrom = "chr7"

# df = bam2matrix(mut_file, pileup=pileup_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
df = tumor2matrix(mut_file, bam=bam_file, chrom=chrom, pon_list=pon_list, EBconfig=EBconfig)
df

## make EBcache matrix
+ to be used by bam2matrix in cache-mode

In [ ]:
from ebcache import PON2matrix

EBconfig = {
    "pon_path": pon_path,
    "bed_file": os.path.join(static, "bed_files/SureSelect/hg38/SS_HAEv7_hg38_Padded.bed"),
    "mawk_path": "../shell",
    "genome_split": f"{static}/genome/gatk/hg38/split",
    "pon_path": os.path.join(testdata, "testpon"),
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"},
    "use_cache": True
}
pon_list = "Pon_chr7.txt"
chrom = "chr7"

EBcache_matrix(pon_list, chrom, EBconfig)

## EBcache matrix2AB
+ second level cache containing the ABparams

In [ ]:
chr7_df = pd.read_csv(os.path.join(pon_path, "matrix/chr7.pon.gz"), compression="gzip", sep="\t")
chr7_df

In [ ]:
from ebcache import PONmatrix2AB_multi

# use this but going through all letters and 
PONEB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunk_size": 1000
}

%time df = PONmatrix2AB_multi(chr7_df[:500000], config=PONEB_config)
# df.to_csv(os.path.join(pon_path, "ABcache/chr7.AB.gz"), sep="\t", compression="gzip", index=False)
df

In [ ]:
%time print("Hello")

## caching the caching
+ trying to find common cachable AB parameters

In [ ]:
# merge AB with matrix
merge = chr7_df[:500000].merge(df, left_index=True, right_index=True)
merge

## flatten the df by the letters

In [ ]:
dfs = []
for letter in "AGCTID":
    print(letter)
    letter_df = merge.loc[:, [f"{letter}_x", "Depth", f"{letter}_y"]].rename({f"{letter}_x":"tumor", f"{letter}_y":"AB"}, axis=1)
    dfs.append(letter_df)
merge2 = pd.concat(dfs)
merge2

## split left and right

In [ ]:
ldfs = []
for l in [0,1]:
    ldf = pd.DataFrame()
    for col in ['tumor', 'Depth', 'AB']:
        ldf[col] = merge2[col].str.split("-").str[0]
    ldfs.append(ldf)
merge3 = pd.concat(ldfs)
merge3

In [ ]:
# merge3.to_csv(os.path.join(pon_path, "chr7.cache.cache.gz"), sep="\t", index=False, compression="gzip")
merge3 = pd.read_csv(os.path.join(pon_path, "chr7.cache.cache.gz"), sep="\t", compression="gzip")

### sort the splits by Depth

In [ ]:
merge3.query('tumor != "0|0|0|0|0"')

In [ ]:
row = merge3.iloc[138]
row

In [ ]:
def order(row):
    dl = [int(d) for d in row['Depth'].split("|")]
    row['Depth'] = "|".join([str(d) for d in sorted(dl)])
    row['tumor'] = "|".join([str(row['tumor'].split("|")[i]) for i in np.argsort(dl)])
    return row

In [ ]:
order(row)

In [ ]:
merge4 = merge3.apply(order, axis=1)

In [ ]:
merge5 = merge4.sort_values(['AB', 'Depth', 'tumor'])
merge5

In [ ]:
merge6 = merge5.groupby(['Depth', 'tumor']).agg(['first', 'last', "count"])
merge6

In [ ]:
merge6['AB']

In [ ]:
merge7 = merge6['AB'].reset_index().sort_values(['count'], ascending=False)
merge7['freq'] = merge7['count'] / 6000000
merge7['sumfreq'] = merge7['freq'].cumsum()
merge7

In [ ]:
merge7.to_csv(os.path.join(pon_path, "chr7.cache.cache.gz"), sep="\t", index=False, compression="gzip")
# merge7 = pd.read_csv(os.path.join(pon_path, "chr7.cache.cache.gz"), sep="\t", compression="gzip")

In [ ]:
merge7.query('tumor == "0|0|0|0|0"')

### EX|INport

In [ ]:
# get the code
import sys
sys.path.append('../code')

# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
ebdata = f"{testdata}/EBdata"
pon_path = f"{testdata}/testpon"
static = f"{home}/Dropbox/Icke/Work/static"


# df.to_csv(f"{ebdata}/002.matrix", sep='\t', index=False)
df = pd.read_csv(f"{ebdata}/002.matrix", sep='\t')
df[:3]

## matrix to AB

### get_count_df for matrix2EB

In [ ]:
from ebutils import get_count_df
row = df.iloc[2,:]
row
cdf = get_count_df(row['PON:ALT=Depth'], EBconfig["count_dict"])
cdf

In [ ]:
from ebconvert import matrix2AB
from ebcore import fit_bb
bb_params = fit_bb(cdf, 0.5)
bb_params

In [ ]:
matrix2AB(row, config=EBconfig)

### expand matrix2AB to df

In [ ]:
df['ABparams'] = df.apply(matrix2AB, config=EBconfig, axis=1)
df[:3]

## AB to EB

In [ ]:
# make a row with the same values as in EBold for comparison
row_dict = {
    "Chr": ["chr7"],
    "Start": 680651,
    "Tumor:Alt=Depth": "0-5=12-42",
    "ABparams": "0.20509|23.07746-0.22661|2.47235"
}

test_row = pd.DataFrame().from_dict(row_dict).iloc[0]
test_row
from ebconvert import AB2EBscore
AB2EBscore(test_row, config=EBconfig)

In [ ]:
df['EBscore'] = df.apply(AB2EBscore, config=EBconfig, axis=1)
df

In [ ]:
ls

In [ ]:
p = pd.DataFrame()

In [ ]:
p.index[0]